In [1]:
from jovis_model.configs.config import Config

In [5]:
c = Config(
    **{
        "command": "train",
        "task": "ynat",
        "output_dir": "abcd",
        "train_config": {
            "m_name": "abcd",
            "config_name": "abcd",
            "tokenizer_name": "abcd",
            "cache_dir": "abcd",
            "encoder_layerdrop": 0.5,
            "decode_layerdrop": 0.5,
            "dropout": 0.5,
            "attention_dropout": 0.5,
            "adafactor": True
        }
    }
)

command='train' task='ynat' output_dir='abcd' gpus=None fp16=None num_sanity_val_steps=2 max_grad_norm=1.0 gradient_accumulation_steps=1 seed=42 metric_key='loss' patience=5 early_stopping_mode='max' train_config={'m_name': 'abcd', 'config_name': 'abcd', 'tokenizer_name': 'abcd', 'cache_dir': 'abcd', 'encoder_layerdrop': 0.5, 'decode_layerdrop': 0.5, 'dropout': 0.5, 'attention_dropout': 0.5, 'adafactor': True}


In [6]:
c

Config(command='train', task='ynat', output_dir='abcd', gpus=None, fp16=None, num_sanity_val_steps=2, max_grad_norm=1.0, gradient_accumulation_steps=1, seed=42, metric_key='loss', patience=5, early_stopping_mode='max', train_config=TrainConfig(m_name='abcd', config_name='abcd', tokenizer_name='abcd', cache_dir='abcd', encoder_layerdrop=0.5, decode_layerdrop=0.5, dropout=0.5, attention_dropout=0.5, learning_rate=5e-05, weight_decay=0.0, adam_epsilon=1e-08, warmup_steps=None, warmup_ratio=None, num_train_epochs=4, adafactor=True, verbose_step_count=100))